# Get Data
The following code block allows a user to get data files and utility scripts beginning with only main.ipynb either locally or on Google Colab.

In [2]:
import os.path
from os import path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
from time import time
from scripts.utils import *
import sklearn
from sklearn.model_selection import train_test_split

try:
    __import__("wget")
    import wget
    print("wget import exists")
    if(not path.exists("data")):
        print("Making Directory data")
        os.mkdir("data")   
    if(not path.exists("data/control.xls")):
        print("Downloading control.xls")
        wget.download("https://raw.githubusercontent.com/smithandrewk/sleep/main/data/control.xls","data/control.xls")
    if(not path.exists("data/deprivation.xls")):
        print("Downloading deprivation.xls")
        wget.download("https://raw.githubusercontent.com/smithandrewk/sleep/main/data/deprivation.xls","data/deprivation.xls")
    if(not path.exists("scripts")):
        print("Making Directory scripts")
        os.mkdir("scripts")   
    if(not path.exists("scripts/utils.py")):
        print("Downloading utils.py")
        wget.download("https://raw.githubusercontent.com/smithandrewk/sleep/main/scripts/utils.py","scripts/utils.py")
except ImportError:
    print("wget import does not exist, using python2.7 wget")
    if(not path.exists("data")):
        print("Making Directory data")
        !mkdir data   
    if(not path.exists("data/control.xls")):
        print("control.xls not in data")
        !wget -O data/control.xls https://raw.githubusercontent.com/smithandrewk/sleep/main/data/control.xls
    if(not path.exists("data/deprivation.xls")):
        print("deprivation.xls not in data")
        !wget -O data/deprivation.xls https://raw.githubusercontent.com/smithandrewk/sleep/main/data/deprivation.xls
    if(not path.exists("scripts")):
        print("Making Directory scripts")
        !mkdir scripts
    if(not path.exists("scripts/utils.py")):
        !wget -O scripts/utils.py https://raw.githubusercontent.com/smithandrewk/sleep/main/scripts/utils.py
control = preprocess("control")
deprivation = preprocess("deprivation")
control_deprivation = pd.concat([control,deprivation])
control_deprivation.to_csv("data/controldeprivation_preprocessed.csv",index=False)
p,s,w = class_count(control)
p,s,w = class_count(deprivation)
p,s,w = class_count(control_deprivation)

wget import exists
Examples:
    Total: 8641
    P: 367 (4.25% of total)
    S: 3965 (45.89% of total)
    W: 4309 (49.87% of total)

Examples:
    Total: 8641
    P: 614 (7.11% of total)
    S: 3294 (38.12% of total)
    W: 4733 (54.77% of total)

Examples:
    Total: 17282
    P: 981 (5.68% of total)
    S: 7259 (42.00% of total)
    W: 9042 (52.32% of total)



In [ ]:
target_filename = 'controldeprivation' # or 'control' or 'deprivation'

In [4]:
df = pd.read_csv("data/"+target_filename+"_preprocessed.csv")

## Balancing
ps = df.loc[df["Class"]==0]
ss = df.loc[df["Class"]==1]
ws = df.loc[df["Class"]==2]
for i in range(int(w/p)):
  df = pd.concat([df,ps])
p,s,w = class_count(df)

df.to_csv("data/"+target_filename+"_balanced.csv",index=False)

Examples:
    Total: 17282
    P: 981 (5.68% of total)
    S: 7259 (42.00% of total)
    W: 9042 (52.32% of total)



In [ ]:
df = pd.read_csv("data/"+target_filename+"_balanced.csv")

# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

total = p + s + w
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_p = (1 / p)*(total)/2.0 
weight_for_w = (1 / w)*(total)/2.0
weight_for_s = (1 / s)*(total)/2.0


class_weight = {0: weight_for_p, 1: weight_for_s, 2: weight_for_w}

print('Weight for class 0: {:.2f}'.format(weight_for_p))
print('Weight for class 1: {:.2f}'.format(weight_for_s))
print('Weight for class 2: {:.2f}'.format(weight_for_w))

In [ ]:
def train(INPUT_FEATURES,hln=20,EPOCHS=100,BATCH_SIZE=200,weights=False):
    model = get_compiled_model(hln,INPUT_FEATURES=INPUT_FEATURES,dropout=True)
    """
    We one-hot encode the targets. Mathematically, this is good for calculating
    loss. CategoricalCrossEntropy simplifies to a negative log when targets are
    one-hot encoded. However, I simply recieved an error from model.fit when I 
    did not one-hot encode.
      @y : targets
      @depth : number of targets
    """
    # Callback choices
    plot_losses = TrainingPlot()
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='categorical_accuracy', 
        verbose=1,
        patience=100,
        mode='max',
        restore_best_weights=True)
    tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
    if(weights):
        baseline_history = model.fit(
            train_features,
            tf.one_hot(train_labels,depth=3),
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(val_features, tf.one_hot(val_labels,depth=3)),
            callbacks=[early_stopping],
            class_weight=class_weight)
    else:
        baseline_history = model.fit(
        train_features,
        tf.one_hot(train_labels,depth=3),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(val_features, tf.one_hot(val_labels,depth=3)),
        callbacks=[early_stopping])
    train_predictions_baseline = model.predict(train_features, batch_size=BATCH_SIZE)
    test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)
    baseline_results = model.evaluate(test_features, tf.one_hot(test_labels,depth=3),
                                      batch_size=BATCH_SIZE, verbose=0)
    plot_cm(tf.one_hot(test_labels,depth=3).numpy().argmax(axis=1),test_predictions_baseline.argmax(axis=1),baseline_results,hln)
    date = strftime('%X %x').replace("/","").split()
    plt.savefig("figures/"+str(date[1])+"@"+str(date[0][:5].replace(":",""))+"_"+str(hln)+".png",bbox_inches='tight')
    return baseline_history,baseline_results,date

In [ ]:
trains = []
tests = []
for i in range(5):
    train_history,test_history,date = train(INPUT_FEATURES=(train_features.shape[-1],),
                                   hln=200,
                                   EPOCHS=1000,
                                   weights=True)
    trains.append(train_history)
    tests.append(test_history)

In [ ]:
accs = [test[1] for test in tests]
print(accs)

In [ ]:
plot_metrics(train_history)
plt.savefig("figures/"+str(date[1])+"@"+str(date[0][:5].replace(":",""))+"_training.png",bbox_inches='tight')